### Load Libraries

In [1]:
%matplotlib inline
from __future__ import print_function
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import LSTM, SpatialDropout1D 
from sklearn.model_selection import train_test_split
import csv
import keras
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import os
#from getcpuinfo import cpuinfo
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import requests
from apscheduler.schedulers.background import BackgroundScheduler
import datetime
from keras import regularizers

Using TensorFlow backend.


### Load Raw Training Data

In [2]:
inpData = []
labels = []

'''
with open('cdata.csv') as csv_file:
    lines = csv.reader(csv_file, delimiter=',')
    for row in lines:
        labels.append(row[0])
        inpData.append(float(row[1]))
'''
inpDataArray = np.loadtxt("usageData.csv",delimiter = ',') 

### Set the Target Labels

In [3]:
minThreshold = 30
maxThreshold = 70
target = []
cpuUsage = []
historyParam = 50
for i in range(0,inpDataArray.shape[1]):
    inpData = inpDataArray[:,i]
    index = historyParam
    while index < len(inpData):
        currRec = inpData[index-historyParam:index]
        if inpData[index] > maxThreshold:# Scale Up
            target.append(1)
        elif inpData[index] < minThreshold: # Scale Down
            target.append(-1)
        else:
            target.append(0)  #Remain same
        cpuUsage.append(currRec)
        index += 1

In [4]:
cpuUsageData = np.expand_dims(np.array(cpuUsage,dtype = 'float'),axis = 2)
cpuScaling = np.array(target,dtype = 'float')
print(cpuScaling.shape)
print(cpuUsageData.shape)

cpuScalingCategorical = to_categorical(cpuScaling, num_classes = 3)

(47072,)
(47072, 50, 1)


### Preprocessing Input Data

In [5]:
'''cpuUsage = []
cpuScaling = []
historyParam = 50
index = historyParam

while index<len(inpData):
    currRec = [[inpData[i]] for i in range(index-historyParam,index)]
    cpuUsage.append(currRec)
    index+=1'''
    

'cpuUsage = []\ncpuScaling = []\nhistoryParam = 50\nindex = historyParam\n\nwhile index<len(inpData):\n    currRec = [[inpData[i]] for i in range(index-historyParam,index)]\n    cpuUsage.append(currRec)\n    index+=1'

In [6]:
'''cpuScaling = target[historyParam-1:len(target)]


cpuUsageData = np.array(cpuUsage,dtype = 'float')
cpuScaling = np.array(cpuScaling,dtype = 'float')

cpuScalingCategorical = to_categorical(cpuScaling, num_classes = 3)'''

"cpuScaling = target[historyParam-1:len(target)]\n\n\ncpuUsageData = np.array(cpuUsage,dtype = 'float')\ncpuScaling = np.array(cpuScaling,dtype = 'float')\n\ncpuScalingCategorical = to_categorical(cpuScaling, num_classes = 3)"

In [7]:
#len(inpData)
X_train, X_test, y_train, y_test = train_test_split(cpuUsageData , cpuScalingCategorical, test_size=0.25, random_state=42)

In [8]:
cpuUsageData.shape
#np.shape(cpuUsage)

(47072, 50, 1)

In [9]:
cpuScalingCategorical.shape

(47072, 3)

In [10]:
model = Sequential()
#model.add(SpatialDropout1D(0.3))
model.add(Flatten(batch_input_shape = (None,50,1)))

model.add(Dense(50, activation = 'relu',
                    kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.1)))

model.add(Dropout(0.3, noise_shape=None, seed=None))

model.add(Dense(25, activation = 'relu',
                    kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.1)))

model.add(Dropout(0.3, noise_shape=None, seed=None))

#model.add(SpatialDropout1D(0.3))
model.add(Dense(12,activation = 'relu',
                    kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.1)))

model.add(Dropout(0.3, noise_shape=None, seed=None))

model.add(Dense(6,activation = 'relu',
                    kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.1)))

model.add(Dropout(0.3, noise_shape=None, seed=None))

model.add(Dense(3, activation='softmax',kernel_initializer='he_normal'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam',metrics = ['accuracy'])
#model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 25)                1275      
_________________________________________________________________
dropout_2 (Dropout)          (None, 25)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 12)                312       
_________________________________________________________________
dropout_3 (Dropout)          (None, 12)                0         
__________

In [12]:
filepath="3OutputDNNV2BestWeights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [13]:
'''if os.path.isfile("3OutputDNNBestWeights.hdf5"):
    #print("Hey")
    model.load_weights("3OutputDNNBestWeights.hdf5")'''

'if os.path.isfile("3OutputDNNBestWeights.hdf5"):\n    #print("Hey")\n    model.load_weights("3OutputDNNBestWeights.hdf5")'

In [15]:
history = model.fit(X_train, y_train, epochs = 25,validation_data = [X_test, y_test],verbose = 1, callbacks = callbacks_list,batch_size = 1000)

Train on 35304 samples, validate on 11768 samples
Epoch 1/25
35304/35304 [==============================] - 0s 4us/step - loss: 0.8405 - acc: 0.7394 - val_loss: 0.7009 - val_acc: 0.8856

Epoch 00001: val_acc improved from 0.88537 to 0.88562, saving model to 3OutputDNNV2BestWeights.hdf5
Epoch 2/25
35304/35304 [==============================] - 0s 4us/step - loss: 0.8049 - acc: 0.7451 - val_loss: 0.6695 - val_acc: 0.8899

Epoch 00002: val_acc improved from 0.88562 to 0.88987, saving model to 3OutputDNNV2BestWeights.hdf5
Epoch 3/25
35304/35304 [==============================] - 0s 4us/step - loss: 0.7726 - acc: 0.7503 - val_loss: 0.6290 - val_acc: 0.8897

Epoch 00003: val_acc did not improve from 0.88987
Epoch 4/25
35304/35304 [==============================] - 0s 4us/step - loss: 0.7453 - acc: 0.7574 - val_loss: 0.5951 - val_acc: 0.8929

Epoch 00004: val_acc improved from 0.88987 to 0.89293, saving model to 3OutputDNNV2BestWeights.hdf5
Epoch 5/25
35304/35304 [============================

In [ ]:
res = model.predict(X_test,batch_size = 1)

In [ ]:
print(res)

In [ ]:
np.argmax(res,axis = 1)

In [ ]:
print(inpData[0+1])